In [4]:
import sys, os
sys.path.append('..')

from src.drone_em_dl import *


from datetime import datetime
import tensorflow as tf

from clearml import Task

In [6]:
data = data.Data()
data.load_data('../data/raw/falster_data_Kristian.csv')
data.get_features([11,12,13,22,23,24,25,26,27,28,29,30,31]) #[1,2,3,4,5,6,7,8,9,10,11,12,13,14]  #[1,2,3,4,5,6,7,11,12,13,14,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31]
data.train_test_split(split=0.8)
data.norm_data()

In [8]:
task = Task.init(project_name="Drone_em_dl", task_name=f"Drone_em_dl_test2")


ClearML Task: created new task id=e6869093c3054370aaea2b279490bf0a
2023-04-07 18:22:31,355 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: https://app.clear.ml/projects/d2611671b215498fb9b50456c1357c66/experiments/e6869093c3054370aaea2b279490bf0a/output/log


ClearML Monitor: GPU monitoring failed getting GPU reading, switching off GPU monitoring


In [9]:

with Fae() as ae:
    ae = ae.make_model(
            input_size=data.norm_data.norm_train[0].shape,
            latent_space_dim=3,
            dense_neurons=[128,128,128,128,128,56,56,28,28],
            dropout_prob=0.1,
            name='test',
        )


2023-04-07 18:22:59.384688: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-04-07 18:22:59.384851: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1 Max

systemMemory: 64.00 GB
maxCacheSize: 24.00 GB



In [10]:
ae.summary()

Model: "test"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 AE_input (InputLayer)       [(None, 13)]              0         
                                                                 
 encoder_model (Functional)  (None, 3)                 167978    
                                                                 
 decoder_model (Functional)  (None, 13)                83993     
                                                                 
Total params: 251,971
Trainable params: 247,123
Non-trainable params: 4,848
_________________________________________________________________


In [11]:
callbacks = get_callbacks(ae)

2023-04-07 18:23:12.602594: I tensorflow/core/profiler/lib/profiler_session.cc:110] Profiler session initializing.
2023-04-07 18:23:12.602611: I tensorflow/core/profiler/lib/profiler_session.cc:125] Profiler session started.
2023-04-07 18:23:12.602792: I tensorflow/core/profiler/lib/profiler_session.cc:143] Profiler session tear down.


In [12]:


model_fit(
        ae,
        data.norm_data.norm_train,
        data.norm_data.norm_train,
        batch_size=200,
        epochs=100,
        verbose=1,
        callbacks=callbacks,
)

Epoch 1/100


2023-04-07 18:23:33.313600: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


InvalidArgumentError: Cannot assign a device for operation test/encoder_model/encoder2_dense_1/MatMul/ReadVariableOp: Could not satisfy explicit device specification '' because the node {{colocation_node test/encoder_model/encoder2_dense_1/MatMul/ReadVariableOp}} was colocated with a group of nodes that required incompatible device '/job:localhost/replica:0/task:0/device:GPU:0'. All available devices [/job:localhost/replica:0/task:0/device:CPU:0, /job:localhost/replica:0/task:0/device:GPU:0]. 
Colocation Debug Info:
Colocation group had the following types and supported devices: 
Root Member(assigned_device_name_index_=2 requested_device_name_='/job:localhost/replica:0/task:0/device:GPU:0' assigned_device_name_='/job:localhost/replica:0/task:0/device:GPU:0' resource_device_name_='/job:localhost/replica:0/task:0/device:GPU:0' supported_device_types_=[CPU] possible_devices_=[]
ResourceApplyAdamWithAmsgrad: CPU 
ReadVariableOp: GPU CPU 
_Arg: GPU CPU 

Colocation members, user-requested d